In [2]:
using  FFTW
using  LinearAlgebra
using  ProgressMeter
using  BenchmarkTools
using  Plots
pyplot()

┌ Info: Recompiling stale cache file /Users/navaro/.julia/compiled/v1.0/Plots/ld3vC.ji for Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1187
┌ Info: Recompiling stale cache file /Users/navaro/.julia/compiled/v1.0/PyPlot/oatAj.ji for PyPlot [d330b81b-6aea-500a-939a-2ce795aea3ee]
└ @ Base loading.jl:1187


Plots.PyPlotBackend()

In [3]:
using FModels

┌ Info: Precompiling FModels [56bd5bc8-d889-11e8-33a6-bb3ace1bc7c7]
└ @ Base loading.jl:1189
┌ Warning: Package FModels does not have Plots in its dependencies:
│ - If you have FModels checked out for development and have
│   added Plots as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with FModels
└ Loading Plots into FModels from project dependency, future warnings for FModels are suppressed.
ERROR: LoadError: LoadError: UndefVarError: Times not defined
Stacktrace:
 [1] top-level scope at none:0
 [2] include at ./boot.jl:317 [inlined]
 [3] include_relative(::Module, ::String) at ./loading.jl:1041
 [4] include at ./sysimg.jl:29 [inlined]
 [5] include(::String) at /Users/navaro/.julia/packages/FModels/PA8E9/src/FModels.jl:1
 [6] top-level scope at none:0
 [7] include at ./boot.jl:317 [inlined]
 [8] include_relative(::Module, ::String) at ./loading.jl:1041
 [9] include(::Module, ::String)

ErrorException: Failed to precompile FModels [56bd5bc8-d889-11e8-33a6-bb3ace1bc7c7] to /Users/navaro/.julia/compiled/v1.0/FModels/mpuzA.ji.

In [4]:

function main(mesh::Mesh, freq::Freq, times::Times, Gamma::Vector{Float64})
    
    function solve!(h, u, times)
                
        prog = Progress(times.nt,1) # progress bar
        
        dh, du = similar(h), similar(u)
        hhat, uhat = similar(h), similar(u)
        hnew, unew = similar(h), similar(u)
        
        Int1 = similar(u)
        Int2 = similar(h)
        Int3 = similar(h)
        
        Px  = plan_fft(hnew; flags = FFTW.MEASURE)

        function fwave!(h::Vector{Complex{Float64}},
                        u::Vector{Complex{Float64}})
             
             ldiv!(hnew, Px , h)
             
             Int1  .= u
             Int1 .*= Gamma
             ldiv!(unew, Px , Int1)
             unew  .^= 2
             mul!(Int1, Px , unew)
             Int1 .*= H
             
             Int2  .= h
             Int2 .*= Dx
             ldiv!(unew, Px , Int2)
             unew  .*= hnew
             mul!(Int2, Px , unew)
             
             Int3  .= h
             Int3 .*= Gamma
             ldiv!(unew, Px, Int3)
             unew  .*= hnew
             mul!(Int3 , Px , unew)
             Int3 .*= H
             
             hnew  .= -u
             hnew .*= Dx
             
             Int1 .-= Int2
             Int1 .-= Int3
             Int1 .*= Pi
             Int1 .*= epsilon
             
             u  .= h
             u .*= H
             u .+= Int1
             
             h .= hnew
             
        end
    
        function rk4!( h::Vector{Complex{Float64}},
                       u::Vector{Complex{Float64}})
            
            hhat .= h
            uhat .= u
            fwave!( hhat, uhat)
            dh .= hhat
            du .= uhat
            
            hhat .= h .+ dt/2*hhat
            uhat .= u .+ dt/2*uhat
            fwave!( hhat,uhat)
            dh .+= 2 * hhat
            du .+= 2 * uhat
            
            hhat .= h .+ dt/2*hhat
            uhat .= u .+ dt/2*uhat
            fwave!( hhat, uhat)
            dh .+= 2 * hhat
            du .+= 2 * uhat
            
            hhat .= h .+ dt*hhat
            uhat .= u .+ dt*uhat
            fwave!( hhat, uhat)
            dh .+= hhat
            du .+= uhat
            
            h .+= dt/6 * dh
            u .+= dt/6 * du
        end
        
        label = "Cheng et al."
        Modeles[label] = []
      
        push!(Modeles[label],(h,u))
        for l in range(1,times.nt-1)
            
            dt = times.t[l+1]-times.t[l]
            
            rk4!( h, u)
        
            push!(Modeles[label],(h,u))   
            next!(prog)
        end
                
    end
    
    H     = -1im * sign.(freq.k)     # Hilbert transform
    Dx    =  1im * freq.k            # Differentiation
    Pi    = Gamma .< freq.kmax * 2/3 # Dealiasing low-pass filter
    
    h     = zeros(Complex{Float64},N)
    h    .= exp.(-mesh.x.^2)
    h    .= Pi .* fft(h)
    u     = zeros(Complex{Float64},N)
    u    .= Pi .* fft(u)
    
    solve!(h, u, times )
    
end

UndefVarError: UndefVarError: Mesh not defined

In [5]:
epsilon = 1/2
N       = 2^12
L       = 10
T       = 5
dt      = 0.001

@show epsilon,N,L,T,dt

mesh  = Mesh(-L, L, N)
freq  = Freq(mesh.dx, N)
times = Times(dt, T)

Gamma   = abs.(freq.k)
Modeles = Dict()
    
@time main(mesh, freq, times, Gamma)
fig(5, times, Gamma, freq, Modeles, epsilon, mesh)

(epsilon, N, L, T, dt) = (0.5, 4096, 10, 5, 0.001)


UndefVarError: UndefVarError: Mesh not defined